In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import tensorflow as tf

COLOR_MODE = "grayscale"
CHANNELS = 1 if COLOR_MODE == "grayscale" else 3
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
TARGET_SIZE = (IMAGE_HEIGHT, IMAGE_WIDTH)
BATCH_SIZE = 32
SEED = 42


TRAINING_PATH = "./dataset/images"
MASK_PATH = "./dataset/mask"
TESTING_PATH = "./dataset/Test_Set/Test_Set"


In [2]:
data_gen_args = dict(featurewise_center=True,
                     featurewise_std_normalization=True,
                     rotation_range=90,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2,
                     validation_split=0.2)

image_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)
mask_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)
seed = 1

# Custom generator function to yield image and mask batches together
def combined_generator(image_generator, mask_generator):
    for img_batch in image_generator:
        mask_batch = next(mask_generator)
        yield img_batch, mask_batch

# Training Generators
image_train_generator = image_datagen.flow_from_directory(
    './dataset/images',
    class_mode=None,
    seed=seed,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    subset='training',
)
mask_train_generator = mask_datagen.flow_from_directory(
    './dataset/masks',
    class_mode=None,
    seed=seed,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    color_mode=COLOR_MODE,
    subset='training',
)
train_generator = combined_generator(image_train_generator, mask_train_generator)


# Validation Generators
image_val_generator = image_datagen.flow_from_directory(
    './dataset/images',
    class_mode=None,
    seed=seed,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    subset='validation',
)
mask_val_generator = mask_datagen.flow_from_directory(
    './dataset/masks',
    class_mode=None,
    seed=seed,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    color_mode=COLOR_MODE,
    subset='validation',
)
val_generator = combined_generator(image_val_generator, mask_val_generator)

print(train_generator)

Found 16933 images belonging to 4 classes.
Found 16933 images belonging to 4 classes.
Found 4232 images belonging to 4 classes.
Found 4232 images belonging to 4 classes.
<generator object combined_generator at 0x0000015BDA618510>


In [3]:
import keras

model = keras.models.Sequential([
    keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS)),

    # preprocessing,

    # keras.layers.Conv2D(32, 3, padding="same", activation="relu"),
    keras.layers.Conv2D(64, 3, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(),
    keras.layers.Dropout(0.2),

    keras.layers.Conv2D(128, 3, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(),
    keras.layers.Dropout(0.2),
    
    keras.layers.Conv2D(256, 3, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(),
    keras.layers.Dropout(0.2),

    keras.layers.Conv2D(512, 3, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(),
    keras.layers.Dropout(0.2),
    

    keras.layers.GlobalAveragePooling2D(),

    # keras.layers.Flatten(),
    # keras.layers.Dropout(0.2),
    # keras.layers.Dense(1024, activation="relu"),
    # keras.layers.Dropout(0.2),
    # keras.layers.Dense(256, activation="relu"),
    # keras.layers.Dropout(0.2),
    # keras.layers.Dense(32, activation="relu"),
    # keras.layers.Dropout(0.2),

    keras.layers.Dense(4, activation="softmax"),
])

# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(), 
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

model.summary(
    expand_nested=True,
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 64)      640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 128, 128, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 128)     73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 128)      0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 64, 64, 128)       0

In [4]:
# import os
# import cv2
# import numpy as np
# from tqdm import tqdm
# from tensorflow.keras.preprocessing.image import load_img, img_to_array

# def load_images_from_directory(directory):
#     images = []
#     masks = []
#     labels = []
#     class_names = os.listdir(directory)
#     # class_names.sort()  # Ensure consistent order of classes

#     for class_name in tqdm(class_names):
#         # class_dir = os.path.join(directory, class_name)
#         class_dir = f"{directory}/{class_name}/images"
#         if os.path.isdir(class_dir):
#             print(class_dir)
#             for image_name in os.listdir(class_dir):
#                 image_path = f"{directory}/{class_name}/images/{image_name}"
#                 mask_path = f"{directory}/{class_name}/masks/{image_name}"
#                 # image = cv2.imread(image_path)
#                 image = load_img(image_path, color_mode=COLOR_MODE, target_size=(IMAGE_HEIGHT, IMAGE_WIDTH), interpolation='bilinear')
#                 # mask = cv2.imread(mask_path)
#                 mask = load_img(mask_path, color_mode=COLOR_MODE, target_size=(IMAGE_HEIGHT, IMAGE_WIDTH), interpolation='bilinear')
#                 if image is not None:
#                     images.append(img_to_array(image)/255.0)
#                     masks.append(img_to_array(mask)/255.0)
#                     labels.append(class_name)
    
#     return images, labels, masks, class_names


# dataset_directory = './dataset'
# images, labels, masks, class_names = load_images_from_directory(dataset_directory)

# def apply_mask(image, mask):
#     masked_image = cv2.bitwise_and(image, mask)
#     return masked_image

# masked_images = []

# for i in range(len(images)):
# 	# mask = create_mask(image.shape)
# 	masked_image = apply_mask(images[i], masks[i])
# 	masked_images.append(masked_image)
# 	# Display or save the masked image
# 	# cv2.imshow('Masked Image', masked_image)
# 	# cv2.waitKey(0)  # Press any key to proceed
# 	# cv2.destroyAllWindows()

# 	# If you want to save the masked images, uncomment the following lines:
# 	# output_dir = 'path/to/save/masked_images'
# 	# os.makedirs(output_dir, exist_ok=True)
# 	# output_path = os.path.join(output_dir, f'masked_{labels[idx]}_{idx}.png')
# 	# cv2.imwrite(output_path, masked_image)
# images = []
# masks = []


In [5]:

# import required module 
import cv2 
# import matplotlib.pyplot as plt 
  
# read image 
# image = cv2.imread('gfg.png') 
  
# call imshow() using plt object 
# plt.imshow(images[0])
# plt.show()
# plt.imshow(masks[0])
# plt.show()
# plt.imshow(masked_images[0])
# plt.show()

# display that image 



# cv2.imshow('Normal Image', images[0])
# cv2.imshow('Mask', masks[0])
# cv2.imshow('Masked Image', masked_images[0])

# import numpy as np

# def randomize_dataset(images, labels):
#     # Generate shuffled indices
#     num_samples = len(images)
#     shuffled_indices = np.random.permutation(num_samples)

#     # Shuffle images and labels
#     shuffled_images = images[shuffled_indices]
#     shuffled_labels = labels[shuffled_indices]

#     return shuffled_images, shuffled_labels

# # Randomize the images in each dataset
# train_ds_images, train_ds_labels = randomize_dataset(train_ds_images, train_ds_labels)
# test_ds_images, test_ds_labels = randomize_dataset(test_ds_images, test_ds_labels)
# validation_ds_images, validation_ds_labels = randomize_dataset(validation_ds_images, validation_ds_labels)

# # Example of randomized dataset
# print("Randomized train dataset:")
# print(train_ds_images.shape)
# print(train_ds_labels.shape)

# # Split data into training and testing sets
# from sklearn.model_selection import train_test_split

# x_train, x_test, y_train, y_test = train_test_split(masked_images, labels, test_size=0.2, random_state=42)

In [6]:
# from sklearn.utils.class_weight import compute_class_weight

# # Compute class weights
# class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

# # Convert class weights to a dictionary format
# class_weights_dict = dict(enumerate(class_weights))

# class_weights_dict

In [7]:
from sklearn.calibration import LabelEncoder
import tensorflow as tf
import numpy as np

# # Train the model
EPOCHS = 100
PATIENCE = 25
earlystop_loss = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True)


# Train the model
history = model.fit_generator(
    train_generator,
    steps_per_epoch=len(image_train_generator),
    epochs=EPOCHS,
    # batch_size=32,
    validation_data=val_generator,
    validation_steps=len(image_val_generator),
    callbacks=[earlystop_loss],
    # class_weight=class_weights_dict,
)

C:\Users\goura\AppData\Local\Temp\ipykernel_1048\3706214970.py:12: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(
c:\Users\goura\anaconda3\envs\tf\lib\site-packages\keras\preprocessing\image.py:1863: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(
c:\Users\goura\anaconda3\envs\tf\lib\site-packages\keras\preprocessing\image.py:1873: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


Epoch 1/100


NotFoundError: Graph execution error:

Detected at node 'sequential/conv2d/Relu' defined at (most recent call last):
    File "c:\Users\goura\anaconda3\envs\tf\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\goura\anaconda3\envs\tf\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\goura\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Users\goura\anaconda3\envs\tf\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\goura\anaconda3\envs\tf\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\goura\anaconda3\envs\tf\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\goura\anaconda3\envs\tf\lib\asyncio\base_events.py", 

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_ds_images, test_ds_labels)
print("Test Accuracy:", test_accuracy)

In [ ]:
from matplotlib import pyplot as plt

print("Epochs run:", len(history.history["loss"]))

acc = history.history["binary_accuracy"]
val_acc = history.history["val_binary_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)

# Train and validation accuracy
plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.ylim((0, 1))
plt.plot(epochs, acc, label="Training accurarcy")
plt.plot(epochs, val_acc, label="Validation accurarcy")
plt.title("Training and Validation accurarcy")
plt.legend()

# Train and validation loss
plt.subplot(1, 2, 2)

plt.plot(epochs, loss, label="Training loss")
plt.plot(epochs, val_loss, label="Validation loss")
plt.title("Training and Validation loss")
plt.legend()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix


# validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
#     TESTING_FOLER,
#     image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
#     batch_size=32,
#     color_mode=COLOR_MODE,
#     shuffle=True,
# )

# Get true labels and predicted probabilities
true_labels = []

predictions = model.predict(test_ds_images)
# Convert probabilities to predicted labels
predicted_labels = np.where(np.array(predictions) > 0.5, 1, 0)

# Convert true labels and predicted labels to numpy arrays
true_labels = np.array(test_ds_labels)
predicted_labels = np.array(predicted_labels)

# Build the confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print(conf_matrix)
# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()
